# **Switch to STAC Catalogue API - OpenSearch Catalogue API Decommissioning**

## **Introduction**

Due to the planned **[decommissioning of the OpenSearch Catalogue API](https://dataspace.copernicus.eu/news/2025-10-16-opensearch-catalogue-api-decommissioning-notice)** (hereafter referred to as Resto) scheduled for **2 February 2026**, users are encouraged to prepare their services and integrations for a transition to the STAC Catalogue API. This Jupyter notebook has been created as a practical migration guide to support that process and to facilitate a smooth and informed transition.

The primary goal of this notebook is to help users understand how existing Resto-based workflows can be translated into equivalent STAC API queries. It provides conceptual guidance as well as concrete examples, enabling users to gradually adapt their applications while preserving existing functionality.

Beyond serving as a replacement for Resto interface, STAC introduces a richer and more expressive data model, along with extended capabilities. By providing a standardized schema to describe spatiotemporal assets, STAC improves data interoperability across platforms and tools. This standardization lowers the entry barrier to efficiently discover, access, and exploit satellite and remote sensing data.

This notebook is structured into three main sections, focusing respectively on:

* Collections & Items,
* Specific Items,
* Filters & Extensions.
  
Each section focuses on practical usage patterns and highlights the correspondence between Resto query parameters and their STAC equivalents, allowing users to incrementally migrate their discovery workflows with confidence.

## **Libraries**

Before we begin, we need to make sure that all the required libraries are installed and imported.

* `pandas`: Python library used for data manipulation and analysis 
* `requests`: Python library used for sending HTTP requests and interacting with APIs 
*  `xml.etree.ElementTree`: Python library used for parsing and navigating an XML document

In [1]:
!pip install pandas
import pandas as pd
import requests
import xml.etree.ElementTree as ET

## **Collections & Items**

This section is dedicated to exploring **collections** - discovering which collections are available, examining the products they contain, and accessing metadata and properties associated with them.  

It also covers **items** within those collections, including how to search for them, retrieve their attributes, and understand their spatial, temporal, and product-specific information.

We will first cover aspects related to Resto and then transition smoothly to STAC.

### **Resto**

Data in Resto are organized into collections, which correspond to different satellites or data sources. Queries can target either all collections or a specific collection. 

Accessing the base Resto URL provides information about products available in the catalogue, rather than details about collections themselves - it does not provide collection-level summaries or structure.

In [2]:
resto_url = (
    "https://catalogue.dataspace.copernicus.eu/resto/api/collections/search.json?"
)

Executing the query below will return metadata about the products in the catalogue. The response includes key fields for each product: type, id, geometry, properties. 

> **Hint:** The `.head()` function is used to display the first five rows of a DataFrame. This is useful for quickly inspecting the structure of your data and verifying that it has been loaded correctly.

In [3]:
json = requests.get(resto_url).json()

print(f"HTTPS Request: {resto_url}")

pd.DataFrame(json["features"]).head()

HTTPS Request: https://catalogue.dataspace.copernicus.eu/resto/api/collections/search.json?


,type,id,geometry,properties
0,Feature,91822f33-b15c-5b60-aa39-6d9f6f5c773b,"{'type': 'Polygon', 'coordinates': [[[-176.587...","{'collection': 'SENTINEL-3', 'status': 'ONLINE..."
1,Feature,bfd8cd54-52a7-48e2-88d0-58df86167399,"{'type': 'Polygon', 'coordinates': [[[-145.885...","{'collection': 'SENTINEL-3', 'status': 'ONLINE..."
2,Feature,eb59e26f-c472-4f5f-8289-49bce7b13b7e,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'collection': 'SENTINEL-3', 'status': 'ONLINE..."
3,Feature,85899f29-0854-30a0-b9a7-5f2d332e6f10,"{'type': 'Polygon', 'coordinates': [[[140.656,...","{'collection': 'SENTINEL-3', 'status': 'ONLINE..."
4,Feature,2b5ae1be-1831-447d-90b0-1e4b4d34d7b2,"{'type': 'Polygon', 'coordinates': [[[138.511,...","{'collection': 'SENTINEL-3', 'status': 'ONLINE..."


The list of available collections must be obtained from the documentation. For the next steps, we will create a Python list containing a few selected collections. This will allow us to perform queries and examples more efficiently.

In [4]:
resto_collections = ["Sentinel1", "Sentinel2", "Sentinel3", "Sentinel5P", "CLMS"]

For each satellite (collection), the set of queryable parameters can be obtained from the resulting XML via **describe.xml** endpoint.

To simplify it, we will create a Python function that reads the relevant parameters for a given collection from the XML. This approach allows users to know in advance what they can query, making it easier to construct valid requests without manually inspecting the XML for every collection.

In [5]:
def parse_resto_collection(collection):
    """
    Fetches and parses the describe.xml metadata for a given Resto collection.

    Args:
        collection (str): The name of the collection (e.g., "Sentinel2").

    Returns:
        dict: A dictionary containing:
            - 'collection': the collection name
            - 'short_name': the short name of the collection
            - 'description': the collection description
            - 'parameters': a list of queryable parameter names available for this collection

    """
    url = f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/{collection}/describe.xml"
    response = requests.get(url)
    root = ET.fromstring(response.content)

    ns = {
        "os": "http://a9.com/-/spec/opensearch/1.1/",
        "param": "http://a9.com/-/spec/opensearch/extensions/parameters/1.0/",
    }

    return {
        "collection": collection,
        "short_name": root.findtext("os:ShortName", namespaces=ns),
        "description": root.findtext("os:Description", namespaces=ns),
        "parameters": [
            p.attrib.get("name") for p in root.findall(".//param:Parameter", ns)
        ],
    }

The code below executes the `parse_resto_collection` function for every collection in the `resto_collections` list.

In [6]:
df_resto = pd.DataFrame([parse_resto_collection(c) for c in resto_collections])
df_resto

,collection,short_name,description,parameters
0,Sentinel1,Sentinel-1,Sentinel-1 Collection,"[maxRecords, index, page, identifier, geometry..."
1,Sentinel2,Sentinel-2,Sentinel-2 Collection,"[maxRecords, index, page, identifier, geometry..."
2,Sentinel3,Sentinel-3,Sentinel-3 Collection,"[maxRecords, index, page, identifier, geometry..."
3,Sentinel5P,Sentinel-5P,Sentinel-5P Collection,"[maxRecords, index, page, identifier, geometry..."
4,CLMS,CLMS,Copernicus Land Monitoring Service Collection,"[maxRecords, index, page, identifier, geometry..."


The code below demonstrates how to retrieve only the **queryable parameters for a specific collection**, in this case Sentinel-2:

In [7]:
s2_info = parse_resto_collection("Sentinel2")
s2_parameters = s2_info["parameters"]
s2_parameters

['maxRecords',
 'index',
 'page',
 'identifier',
 'geometry',
 'box',
 'lon',
 'lat',
 'radius',
 'startDate',
 'completionDate',
 'productIdentifier',
 'productType',
 'tileId',
 'processingLevel',
 'platform',
 'instrument',
 'orbitNumber',
 'sensorMode',
 'cloudCover',
 'updated',
 'publishedAfter',
 'publishedBefore',
 'sortParam',
 'sortOrder',
 'status',
 'exactCount',
 'orbitDirection',
 'relativeOrbitNumber',
 'processingBaseline',
 'missionTakeId']

The base Resto URL allows querying items from all collections. However, if we want to **retrieve products from a specific collection**, we need to include the collection name in the request URL.

In [8]:
collection_name = "CLMS"
resto_url = f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/{collection_name}/search.json"

json = requests.get(resto_url).json()

print(f"HTTPS Request: {resto_url}")

df_features = pd.DataFrame(json["features"])
df_features.head()

HTTPS Request: https://catalogue.dataspace.copernicus.eu/resto/api/collections/CLMS/search.json


,type,id,geometry,properties
0,Feature,000bcf77-ddcf-4fba-b559-d4caaa972e96,"{'type': 'Point', 'coordinates': [67.04, 40.87]}","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."
1,Feature,0018595d-a646-44ed-9b44-b32b0acf2cc1,"{'type': 'Point', 'coordinates': [-87.0, 44.0]}","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."
2,Feature,0029ff2f-67ed-4d14-bc45-7486c03af258,"{'type': 'Point', 'coordinates': [13.29, 58.81]}","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."
3,Feature,003beb57-788a-4ed1-b4f2-440fba282b07,"{'type': 'Point', 'coordinates': [-62.8, 7.41]}","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."
4,Feature,00684216-d9a5-4cc1-9944-e0bba63a09d5,"{'type': 'Point', 'coordinates': [28.0, -16.93]}","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."


In Resto, each feature contains a nested properties dictionary with detailed metadata.

In [9]:
df_props = pd.json_normalize(df_features["properties"])
df_props.head()

,collection,status,parentIdentifier,title,description,organisationName,startDate,completionDate,productType,processingLevel,...,license.grantedOrganizationCountries,license.grantedFlags,license.viewService,license.signatureQuota,license.description.shortName,centroid.type,centroid.coordinates,services.download.url,services.download.mimeType,services.download.size
0,CLMS,ONLINE,None,c_gls_WL_202509220600_1300000000138_ALTI_V2.2....,Copernicus program priorities are to gain from...,None,1995-06-02T15:31:00.000000Z,2025-09-22T06:00:00.000000Z,river_and_lake_water_level,None,...,None,None,public,-1,No license,Point,"[67.04, 40.87]",https://catalogue.dataspace.copernicus.eu/down...,application/geo+json,281916
1,CLMS,ONLINE,None,c_gls_WL_202510170900_1300000000006_ALTI_V2.2....,Copernicus program priorities are to gain from...,None,1992-09-27T02:35:00.000000Z,2025-10-17T09:00:00.000000Z,river_and_lake_water_level,None,...,None,None,public,-1,No license,Point,"[-87.0, 44.0]",https://catalogue.dataspace.copernicus.eu/down...,application/geo+json,688083
2,CLMS,ONLINE,None,c_gls_WL_202509260506_1300000000105_ALTI_V2.2....,Copernicus program priorities are to gain from...,None,1992-09-28T22:30:00.000000Z,2025-09-26T05:06:00.000000Z,river_and_lake_water_level,None,...,None,None,public,-1,No license,Point,"[13.29, 58.81]",https://catalogue.dataspace.copernicus.eu/down...,application/geo+json,373184
3,CLMS,ONLINE,None,c_gls_WL_202505171602_1300000000073_ALTI_V2.2....,Copernicus program priorities are to gain from...,None,1992-09-30T00:51:00.000000Z,2025-05-17T16:02:00.000000Z,river_and_lake_water_level,None,...,None,None,public,-1,No license,Point,"[-62.8, 7.41]",https://catalogue.dataspace.copernicus.eu/down...,application/geo+json,182737
4,CLMS,ONLINE,None,c_gls_WL_202507122311_1300000000172_ALTI_V2.2....,Copernicus program priorities are to gain from...,None,1992-09-26T00:14:00.000000Z,2025-07-12T23:11:00.000000Z,river_and_lake_water_level,None,...,None,None,public,-1,No license,Point,"[28.0, -16.93]",https://catalogue.dataspace.copernicus.eu/down...,application/geo+json,355715


### **STAC**

STAC is built around a set of interconnected JSON objects. STAC catalogue consists of four core resource types: **Catalog**, **Collection**, **Item** and **Asset**. 

Catalogs and Collections are used to define shared metadata for groups of Items, while Items represent individual products, specifying the metadata values and attributes that distinguish one product from another within the same collection. It also enables users to access detailed Asset-level metadata, including spatial information, temporal coverage, and data specifications.

#### **Collections Endpoint**

STAC provides a dedicated endpoint that allows users to retrieve **information about all collections available within the catalogue**. This endpoint returns metadata for each collection.

In [10]:
stac_url = "https://stac.dataspace.copernicus.eu/v1/collections"

In [11]:
json = requests.get(stac_url).json()

print(f"HTTPS Request: {stac_url}")

pd.DataFrame(json["collections"]).head()

HTTPS Request: https://stac.dataspace.copernicus.eu/v1/collections


,id,type,links,title,assets,extent,license,keywords,providers,summaries,...,auth:schemes,sci:citation,stac_version,stac_extensions,storage:schemes,bands,sci:doi,ceosard:type,ceosard:specification,ceosard:specification_version
0,sentinel-3-sl-2-aod-nrt,Collection,"[{'rel': 'items', 'type': 'application/geo+jso...",Sentinel-3 SLSTR Aerosol Optical Depth (NRT),{'thumbnail': {'href': 'https://s3.waw3-2.clou...,"{'spatial': {'bbox': [[-180, -90, 180, 90]]}, ...",other,"[Sentinel, Copernicus, ESA, Satellite, Global,...",[{'url': 'https://sentinels.copernicus.eu/web/...,"{'gsd': [9500], 'platform': ['sentinel-3a', 's...",...,"{'s3': {'type': 's3'}, 'oidc': {'type': 'openI...",Copernicus Sentinel data [Year],1.1.0,[https://stac-extensions.github.io/alternate-a...,"{'cdse-s3': {'type': 'custom-s3', 'title': 'Co...",NaN,NaN,NaN,NaN,NaN
1,sentinel-1-global-mosaics,Collection,"[{'rel': 'items', 'type': 'application/geo+jso...",Sentinel-1 Global Mosaics,{'thumbnail': {'href': 'https://s3.waw3-2.clou...,"{'spatial': {'bbox': [[-180, -90, 180, 90]]}, ...",other,"[Sentinel, Copernicus, ESA, GRD, SAR, C-Band, ...",[{'url': 'https://sentinels.copernicus.eu/web/...,"{'gsd': [20, 40], 'instruments': ['sar'], 'pro...",...,"{'s3': {'type': 's3'}, 'oidc': {'type': 'openI...",Copernicus Sentinel data [Year],1.1.0,[https://stac-extensions.github.io/authenticat...,"{'cdse-s3': {'type': 'custom-s3', 'title': 'Co...",NaN,NaN,NaN,NaN,NaN
2,sentinel-3-olci-2-wfr-nrt,Collection,"[{'rel': 'items', 'type': 'application/geo+jso...",Sentinel-3 OLCI Water Full Resolution (NRT),{'thumbnail': {'href': 'https://s3.waw3-2.clou...,"{'spatial': {'bbox': [[-180, -90, 180, 90]]}, ...",other,"[Sentinel, Copernicus, ESA, Satellite, Global,...",[{'url': 'https://sentinels.copernicus.eu/web/...,"{'gsd': [300], 'platform': ['sentinel-3a', 'se...",...,"{'s3': {'type': 's3'}, 'oidc': {'type': 'openI...",Copernicus Sentinel data [Year],1.1.0,[https://stac-extensions.github.io/eo/v2.0.0/s...,"{'cdse-s3': {'type': 'custom-s3', 'title': 'Co...","[{'name': 'Oa01', 'description': 'Aerosol corr...",NaN,NaN,NaN,NaN
3,sentinel-5p-l2-ch4-nrti,Collection,"[{'rel': 'items', 'type': 'application/geo+jso...",Sentinel-5P Level 2 Methane (NRTI),{'thumbnail': {'href': 'https://s3.waw3-2.clou...,"{'spatial': {'bbox': [[-180, -90, 180, 90]]}, ...",other,"[7000m, Atmosphere, CH4, Copernicus, EC, ESA, ...",[{'url': 'https://sentinel.esa.int/web/sentine...,"{'gsd': [7000], 'platform': ['sentinel-5p'], '...",...,"{'s3': {'type': 's3'}, 'oidc': {'type': 'openI...",Copernicus Sentinel data [Year],1.1.0,[https://stac-extensions.github.io/alternate-a...,"{'cdse-s3': {'type': 'custom-s3', 'title': 'Co...",NaN,NaN,NaN,NaN,NaN
4,sentinel-1-slc-wv,Collection,"[{'rel': 'items', 'type': 'application/geo+jso...",Sentinel-1 Single Look Complex: WV,{'thumbnail': {'href': 'https://s3.waw3-2.clou...,"{'spatial': {'bbox': [[-180, -90, 180, 90]]}, ...",other,"[Sentinel, Copernicus, ESA, SLC, SAR, C-Band, ...",[{'url': 'https://sentinel.esa.int/web/sentine...,"{'platform': ['sentinel-1a', 'sentinel-1b', 's...",...,"{'s3': {'type': 's3'}, 'oidc': {'type': 'openI...",Copernicus Sentinel data [Year],1.1.0,[https://stac-extensions.github.io/authenticat...,"{'cdse-s3': {'type': 'custom-s3', 'title': 'Co...",NaN,NaN,NaN,NaN,NaN


Querying the STAC collections endpoint allows users to obtain any available information related to the properties of each collection.

In this example, we store the following details: id, title and summaries. This approach provides a structured overview of all collections in the catalogue, making it easier to explore their attributes before performing item-level queries.

In [12]:
json = requests.get(stac_url).json()

print(f"HTTPS Request: {stac_url}")

pd.DataFrame(json["collections"])[["id", "title", "summaries"]].head()

HTTPS Request: https://stac.dataspace.copernicus.eu/v1/collections


,id,title,summaries
0,sentinel-3-sl-2-aod-nrt,Sentinel-3 SLSTR Aerosol Optical Depth (NRT),"{'gsd': [9500], 'platform': ['sentinel-3a', 's..."
1,sentinel-1-global-mosaics,Sentinel-1 Global Mosaics,"{'gsd': [20, 40], 'instruments': ['sar'], 'pro..."
2,sentinel-3-olci-2-wfr-nrt,Sentinel-3 OLCI Water Full Resolution (NRT),"{'gsd': [300], 'platform': ['sentinel-3a', 'se..."
3,sentinel-5p-l2-ch4-nrti,Sentinel-5P Level 2 Methane (NRTI),"{'gsd': [7000], 'platform': ['sentinel-5p'], '..."
4,sentinel-1-slc-wv,Sentinel-1 Single Look Complex: WV,"{'platform': ['sentinel-1a', 'sentinel-1b', 's..."


The code below allows us to obtain the **`id`s of all collections available in the catalogue**:

In [13]:
stac_collections = pd.DataFrame(json["collections"])[["id"]]
stac_collections

,id
0,sentinel-3-sl-2-aod-nrt
1,sentinel-1-global-mosaics
2,sentinel-3-olci-2-wfr-nrt
3,sentinel-5p-l2-ch4-nrti
4,sentinel-1-slc-wv
...,...
135,sentinel-5p-l2-o3-nrti
136,sentinel-5p-l2-o3-offl
137,sentinel-5p-l2-so2-offl
138,sentinel-5p-l2-so2-nrti


The code below retrieves **information for a specific collection**:

In [14]:
collection_id = "sentinel-2-l2a"

stac_url = f"https://stac.dataspace.copernicus.eu/v1/collections/{collection_id}"

json = requests.get(stac_url).json()

print(f"HTTPS Request: {stac_url}")

df = pd.DataFrame([json])
df

HTTPS Request: https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a


,id,type,bands,links,title,assets,extent,license,sci:doi,keywords,...,description,item_assets,auth:schemes,ceosard:type,sci:citation,stac_version,stac_extensions,storage:schemes,ceosard:specification,ceosard:specification_version
0,sentinel-2-l2a,Collection,"[{'gsd': 60, 'name': 'B01', 'description': 'Co...","[{'rel': 'items', 'type': 'application/geo+jso...",Sentinel-2 Level-2A,{'thumbnail': {'href': 'https://s3.waw3-2.clou...,"{'spatial': {'bbox': [[-180, -90, 180, 90]]}, ...",other,10.5270/S2_-znk9xsj,"[Copernicus, Sentinel, EU, ESA, Satellite, Glo...",...,The Sentinel-2 Level-2A Collection 1 product p...,"{'AOT_10m': {'gsd': 10, 'type': 'image/jp2', '...","{'s3': {'type': 's3'}, 'oidc': {'type': 'openI...",optical,Copernicus Sentinel data [Year],1.1.0,[https://stac-extensions.github.io/eo/v2.0.0/s...,"{'cdse-s3': {'type': 'custom-s3', 'title': 'Co...",SR,5.0.1


#### **Items Endpoint**

STAC provides a dedicated endpoint that allows users to retrieve **information about items available within the collection**.

The code below retrieves items (individual products) from a specific STAC collection:

In [15]:
collection_id = "sentinel-2-l2a"

stac_url = f"https://stac.dataspace.copernicus.eu/v1/collections/{collection_id}/items"

json = requests.get(stac_url).json()

print(f"HTTPS Request: {stac_url}")

df_features = pd.DataFrame(json["features"])
df_features.head()

HTTPS Request: https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items


,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S2B_MSIL2A_20251231T051319_N0511_R133_T31CEJ_2...,"[2.998598727270369, -82.62787974508461, 3.5898...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[2.998682...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:23:05.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
1,S2B_MSIL2A_20251231T051319_N0511_R133_T31CDM_2...,"[-2.277998803053058, -80.22883981478951, 0.184...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-2.18323...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:23:23.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
2,S2B_MSIL2A_20251231T051319_N0511_R133_T31CDL_2...,"[-2.803841979337122, -81.13320280779526, 1.437...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[0.108578...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:30:58.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
3,S2B_MSIL2A_20251231T051319_N0511_R133_T31CDK_2...,"[-3.447304707739892, -82.03781529080683, 2.739...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[1.369168...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:31:31.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
4,S2B_MSIL2A_20251231T051319_N0511_R133_T30CWS_2...,"[-3.001058129404774, -80.25242038514605, 0.184...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-3.00103...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:24:46.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0


In STAC, each item contains a nested properties dictionary with detailed metadata.

In [16]:
df_props = pd.json_normalize(df_features["properties"])
df_props.head(5)

,gsd,created,expires,updated,datetime,platform,grid:code,published,instruments,end_datetime,...,storage:schemes.cdse-s3.title,storage:schemes.cdse-s3.platform,storage:schemes.cdse-s3.description,storage:schemes.cdse-s3.requester_pays,storage:schemes.creodias-s3.type,storage:schemes.creodias-s3.title,storage:schemes.creodias-s3.platform,storage:schemes.creodias-s3.description,storage:schemes.creodias-s3.requester_pays,processing:software.eometadatatool
0,10,2025-12-31T07:23:05.000000Z,9999-01-01T00:00:00.000000Z,2025-12-31T07:25:38.002693Z,2025-12-31T05:13:19.024000Z,sentinel-2b,MGRS-31CEJ,2025-12-31T07:25:38.002693Z,[msi],2025-12-31T05:13:19.024000Z,...,Copernicus Data Space Ecosystem S3,https://eodata.dataspace.copernicus.eu,This endpoint provides access to EO data which...,False,custom-s3,CREODIAS S3,https://eodata.cloudferro.com,Comprehensive Earth Observation Data (EODATA) ...,True,251021130925+dirty
1,10,2025-12-31T07:23:23.000000Z,9999-01-01T00:00:00.000000Z,2025-12-31T07:26:10.381080Z,2025-12-31T05:13:19.024000Z,sentinel-2b,MGRS-31CDM,2025-12-31T07:26:10.381080Z,[msi],2025-12-31T05:13:19.024000Z,...,Copernicus Data Space Ecosystem S3,https://eodata.dataspace.copernicus.eu,This endpoint provides access to EO data which...,False,custom-s3,CREODIAS S3,https://eodata.cloudferro.com,Comprehensive Earth Observation Data (EODATA) ...,True,251021130925+dirty
2,10,2025-12-31T07:30:58.000000Z,9999-01-01T00:00:00.000000Z,2025-12-31T07:33:41.761622Z,2025-12-31T05:13:19.024000Z,sentinel-2b,MGRS-31CDL,2025-12-31T07:33:41.761622Z,[msi],2025-12-31T05:13:19.024000Z,...,Copernicus Data Space Ecosystem S3,https://eodata.dataspace.copernicus.eu,This endpoint provides access to EO data which...,False,custom-s3,CREODIAS S3,https://eodata.cloudferro.com,Comprehensive Earth Observation Data (EODATA) ...,True,251021130925+dirty
3,10,2025-12-31T07:31:31.000000Z,9999-01-01T00:00:00.000000Z,2025-12-31T07:34:43.634267Z,2025-12-31T05:13:19.024000Z,sentinel-2b,MGRS-31CDK,2025-12-31T07:34:43.634267Z,[msi],2025-12-31T05:13:19.024000Z,...,Copernicus Data Space Ecosystem S3,https://eodata.dataspace.copernicus.eu,This endpoint provides access to EO data which...,False,custom-s3,CREODIAS S3,https://eodata.cloudferro.com,Comprehensive Earth Observation Data (EODATA) ...,True,251021130925+dirty
4,10,2025-12-31T07:24:46.000000Z,9999-01-01T00:00:00.000000Z,2025-12-31T07:27:44.406225Z,2025-12-31T05:13:19.024000Z,sentinel-2b,MGRS-30CWS,2025-12-31T07:27:44.406225Z,[msi],2025-12-31T05:13:19.024000Z,...,Copernicus Data Space Ecosystem S3,https://eodata.dataspace.copernicus.eu,This endpoint provides access to EO data which...,False,custom-s3,CREODIAS S3,https://eodata.cloudferro.com,Comprehensive Earth Observation Data (EODATA) ...,True,251021130925+dirty


#### **Queryables Endpoint**

STAC provides a special queryables endpoint that allows users to explore the catalogue or a specific collection in terms of the **metadata fields available for querying**.

This endpoint returns information about each queryable property, including its data type, description, and constraints, which helps users construct valid and efficient queries without manually inspecting individual items.

By using the queryables endpoint, you can quickly understand which fields can be used in filters and searches across the catalog or within a specific collection.

To access **queryable attributes across the entire catalogue**:

In [17]:
stac_url = "https://stac.dataspace.copernicus.eu/v1/queryables"

json = requests.get(stac_url).json()

print(f"HTTPS Request: {stac_url}")

df = pd.DataFrame([json])
df

HTTPS Request: https://stac.dataspace.copernicus.eu/v1/queryables


,$id,type,title,$schema,properties,additionalProperties
0,https://stac.dataspace.copernicus.eu/v1/querya...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,"{'id': {'type': 'string', 'title': 'Item ID', ...",True


To access its properties:

In [18]:
df_props = pd.json_normalize(df["properties"])
df_props.transpose()

,0
id.type,string
id.title,Item ID
id.minLength,1
id.description,Item identifier
datetime.type,string
datetime.title,Acquired
datetime.format,date-time
datetime.pattern,(\+00:00|Z)$
datetime.description,Datetime
geometry.$ref,https://geojson.org/schema/Feature.json


To access **queryables for a specific collection**:

In [19]:
collection_id = "sentinel-2-l2a"

stac_url = (
    f"https://stac.dataspace.copernicus.eu/v1/collections/{collection_id}/queryables"
)

json = requests.get(stac_url).json()

print(f"HTTPS Request: {stac_url}")

df = pd.DataFrame([json])
df

HTTPS Request: https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/queryables


,$id,type,title,$schema,properties,additionalProperties
0,https://stac.dataspace.copernicus.eu/v1/collec...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,"{'id': {'type': 'string', 'title': 'Item ID', ...",True


To access its properties:

In [20]:
df_props = pd.json_normalize(df["properties"])
df_props = df_props.transpose()
df_props.columns = [f"{collection_id}"]
df_props

,sentinel-2-l2a
id.type,string
id.title,Item ID
id.minLength,1
id.description,Item identifier
datetime.type,string
datetime.title,Acquired
datetime.format,date-time
datetime.pattern,(\+00:00|Z)$
datetime.description,Datetime
geometry.$ref,https://geojson.org/schema/Feature.json


## **Specific Items**

This section is dedicated to retrieving and exploring **individual items** within a collection, including accessing their metadata, properties, spatial and temporal information, and associated assets.

### **Resto**

When using the Resto interface, it is not possible to query a specific product directly by its `id`. Resto only allows searches based on collections and queryable parameters (e.g., `productType`, `cloudCover`, `processingLevel`).

### **STAC**

Unlike Resto, **STAC allows you to search for a specific product using its unique `id`**. You can query the collection and retrieve the full metadata of that item.

In [21]:
item_id = "S2C_MSIL2A_20251229T094421_N0511_R036_T35ULV_20251229T111511"
collection_id = "sentinel-2-l2a"

stac_url = f"https://stac.dataspace.copernicus.eu/v1/collections/{collection_id}/items/{item_id}"

json = requests.get(stac_url).json()

print(f"HTTPS Request: {stac_url}")

df = pd.DataFrame([json])
df.transpose()

HTTPS Request: https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2C_MSIL2A_20251229T094421_N0511_R036_T35ULV_20251229T111511


,0
id,S2C_MSIL2A_20251229T094421_N0511_R036_T35ULV_2...
bbox,"[23.940406316632895, 53.123645037009204, 25.45..."
type,Feature
links,"[{'rel': 'collection', 'type': 'application/js..."
assets,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...
geometry,"{'type': 'Polygon', 'coordinates': [[[25.45795..."
collection,sentinel-2-l2a
properties,"{'gsd': 10, 'created': '2025-12-29T12:00:34.00..."
stac_extensions,[https://cs-si.github.io/eopf-stac-extension/v...
stac_version,1.1.0


To access its properties:

In [22]:
df_features = pd.DataFrame([json["properties"]])
df_features.transpose()

,0
gsd,10
created,2025-12-29T12:00:34.000000Z
expires,9999-01-01T00:00:00.000000Z
updated,2025-12-29T12:07:04.465575Z
datetime,2025-12-29T09:44:21.025000Z
platform,sentinel-2c
grid:code,MGRS-35ULV
published,2025-12-29T12:04:27.641533Z
statistics,"{'water': 0.009615, 'nodata': 28.0182, 'dark_a..."
instruments,[msi]


STAC items may contain nested metadata within the properties field. For example, statistics or other detailed attributes may be stored in sub-dictionaries. The code below demonstrates how to extract  such nested information:

In [23]:
df_stats = pd.json_normalize(json["properties"]["statistics"])
df_stats.transpose()

,0
water,0.009615
nodata,28.018200
dark_area,0.003554
vegetation,0.000000
thin_cirrus,0.001378
cloud_shadow,0.357159
unclassified,0.010159
not_vegetated,0.001065
high_proba_clouds,91.287762
medium_proba_clouds,8.329306


To access geometry:

In [24]:
df_geometry = pd.DataFrame(json["geometry"]["coordinates"][0], columns=["lon", "lat"])
df_geometry

,lon,lat
0,25.457953,54.137210
1,24.891915,53.139700
2,24.010911,53.123645
3,23.940406,54.109206
4,25.457953,54.137210


In STAC, each item contains **assets** that describes the actual data files associated with the product. Assets include URLs to the data, file type, roles, and other metadata.

In [25]:
df_assets = pd.DataFrame([json["assets"]])
df_assets.transpose()

,0
AOT_10m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
AOT_20m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
AOT_60m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
B01_20m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
B01_60m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
B02_10m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
B02_20m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
B02_60m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
B03_10m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...
B03_20m,{'href': 's3://eodata/Sentinel-2/MSI/L2A/2025/...


## **Filters & Extensions**

### **Resto**

Resto does not provide an advanced filtering syntax. Queries are constructed by combining URL parameters.

**Filtering** can be done by attributes, spatial extent, or datetime, and you can include **sorting** options or **limits** on the number of items returned (default is 20). Single values use `param=value`, ranges use `param=[minvalue,maxvalue]`, and multiple parameters are joined with `&`.

Because very general queries may return a large number of results, it is recommended to restrict queries geographically or temporally to avoid exceeding the maximum of 200,000 items. For long date ranges, splitting queries by year or smaller intervals helps maintain performance. **Sorting** is controlled using `sortParam` (e.g., `startDate`, `completionDate`, `published`, `updated`) and `sortOrder` (`ascending` or `descending`).

Some parameters are satellite-specific, such as `cloudCover` (for optical satellites) or `polarisation` (for radar satellites), while others like `instrument` or `productType` depend on the satellite or service. Note that certain features, such as `cloudCover`, may be missing or ambiguous in some products, so filtering by them requires caution.

For full details on all parameters and usage, see the [OpenSearch Catalogue API documentation](https://documentation.dataspace.copernicus.eu/APIs/OpenSearch.html).

In [26]:
collection_name = "CLMS"
product_type = "soil_water_index"

resto_url = f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/{collection_name}/search.json?productType={product_type}"

json = requests.get(resto_url).json()

print(f"HTTPS Request: {resto_url}")

df = pd.DataFrame.from_dict(json["features"])
df.head()

HTTPS Request: https://catalogue.dataspace.copernicus.eu/resto/api/collections/CLMS/search.json?productType=soil_water_index


,type,id,geometry,properties
0,Feature,1a63419d-3369-4655-89ad-c372852728f1,"{'type': 'Polygon', 'coordinates': [[[-180.0, ...","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."
1,Feature,c1caf30e-8348-43b6-b693-d2a5b4b213a9,"{'type': 'Polygon', 'coordinates': [[[-180.0, ...","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."
2,Feature,23f992f3-f12e-4eda-9634-4d56950aeb14,"{'type': 'Polygon', 'coordinates': [[[-180.0, ...","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."
3,Feature,d1bfa68f-1fc9-434a-952e-5e3ff96f6d34,"{'type': 'Polygon', 'coordinates': [[[-180.0, ...","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."
4,Feature,fe327b4a-2af9-4fc0-9645-df42020e6924,"{'type': 'Polygon', 'coordinates': [[[-49.9790...","{'collection': 'CLMS', 'status': 'ONLINE', 'li..."


#### **Examples**

Knowing this, we can create a simple function that follows this structure and focuses specifically on filtering parameters. This function can help streamline queries by automatically constructing the URL with the desired attributes, spatial or temporal constraints, and sorting options.

In [27]:
def resto_search(collection, filters=None):
    """
    Executes a query to the OpenSearch Catalogue API for a given collection.

    Args:
        collection (str): The name of the collection, e.g., "Sentinel2".
        filters (dict, optional): A dictionary of filtering parameters, for example:
            {
                "cloudCover": "[0,10]",
                "startDate": "2021-06-21T00:00:00Z",
                "completionDate": "2021-09-22T23:59:59Z",
                "lon": 21.01,
                "lat": 52.22
            }
            These parameters are translated into URL query parameters.

    Returns:
        pd.DataFrame: A DataFrame containing all the fields returned for each feature
                      in the collection, including `type`, `id`, `geometry`, and `properties`.
                      If no items are found, returns an empty DataFrame.
    """
    base_url = f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/{collection}/search.json"

    filters = filters or {}

    response = requests.get(base_url, params=filters)
    json = response.json()
    features = json.get("features", [])

    df = pd.DataFrame.from_dict(json["features"])

    return df, response.url

The code below executes a query to retrieve twenty Sentinel-2 products acquired in January 2025, sorted by `startDate`:

In [28]:
filters = {
    "startDate": "2025-01-01T00:00:00Z",
    "completionDate": "2025-01-31T23:59:59Z",
    "sortParam": "startDate",
    "maxRecords": "20",
}

df, resto_url = resto_search("Sentinel2", filters)

print(f"HTTPS Request: {resto_url}")

df

HTTPS Request: https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?startDate=2025-01-01T00%3A00%3A00Z&completionDate=2025-01-31T23%3A59%3A59Z&sortParam=startDate&maxRecords=20


,type,id,geometry,properties
0,Feature,049162f6-29eb-4ae2-9936-d4f4ea5cd0d4,"{'type': 'Polygon', 'coordinates': [[[158.1858...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
1,Feature,05c036dd-d6fb-4cf8-a2a8-75d02aa965c9,"{'type': 'Polygon', 'coordinates': [[[158.0939...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
2,Feature,05c92b08-d07a-4f29-9199-809cf3016791,"{'type': 'Polygon', 'coordinates': [[[157.2826...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
3,Feature,0a319264-4bb4-4157-af83-0839dd1a3d12,"{'type': 'Polygon', 'coordinates': [[[158.1858...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
4,Feature,0ad4ff10-1bfb-4f9f-88c5-fcf1406c35d3,"{'type': 'Polygon', 'coordinates': [[[158.1858...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
5,Feature,0c442108-83f6-4c85-bc60-7e8743ce434e,"{'type': 'Polygon', 'coordinates': [[[156.2813...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
6,Feature,12db9bc3-a33d-4f51-ad55-0b0350d75549,"{'type': 'Polygon', 'coordinates': [[[156.2833...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
7,Feature,1abd0faf-5dab-4bdf-950b-9589a80e1430,"{'type': 'Polygon', 'coordinates': [[[157.1880...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
8,Feature,24e5c8a4-32ce-4011-8665-37aabccd5a11,"{'type': 'Polygon', 'coordinates': [[[158.0939...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
9,Feature,313ee18d-74ff-476e-b8cd-0eab9e5930e2,"{'type': 'Polygon', 'coordinates': [[[157.2826...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."


To get the next twenty items from the same query, you can add the `page=2` parameter:

In [29]:
filters = {
    "startDate": "2025-01-01T00:00:00Z",
    "completionDate": "2025-01-31T23:59:59Z",
    "sortParam": "startDate",
    "maxRecords": "20",
    "page": "2",
}

df, resto_url = resto_search("Sentinel2", filters)

print(f"HTTPS Request: {resto_url}")

df

HTTPS Request: https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?startDate=2025-01-01T00%3A00%3A00Z&completionDate=2025-01-31T23%3A59%3A59Z&sortParam=startDate&maxRecords=20&page=2


,type,id,geometry,properties
0,Feature,cbff9b4a-f98d-4c6d-ae0d-bba181b35573,"{'type': 'Polygon', 'coordinates': [[[156.6984...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
1,Feature,d0f31bf8-0e19-4b78-b7b5-dfcb0367b535,"{'type': 'Polygon', 'coordinates': [[[156.6984...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
2,Feature,d198f91e-3e48-4ddf-bd63-725d87400af1,"{'type': 'Polygon', 'coordinates': [[[156.2813...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
3,Feature,e5800cc5-755f-4b71-ab39-75ba2139caa5,"{'type': 'Polygon', 'coordinates': [[[158.2868...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
4,Feature,f9c80d04-202f-40a0-8e8f-bb9093bfd8a7,"{'type': 'Polygon', 'coordinates': [[[157.7636...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
5,Feature,fc1324de-6cee-410e-86a1-90de1350a924,"{'type': 'Polygon', 'coordinates': [[[158.3183...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
6,Feature,03853cbf-bee1-4738-9169-7074e927c97f,"{'type': 'Polygon', 'coordinates': [[[157.1290...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
7,Feature,0d99a056-d390-4297-8049-f243c9fcb555,"{'type': 'Polygon', 'coordinates': [[[157.1290...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
8,Feature,1e6a45d3-fc4a-43d1-b59f-70acccc1bac7,"{'type': 'Polygon', 'coordinates': [[[154.6712...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
9,Feature,2467d219-a97c-4533-b616-cb27ad6945ef,"{'type': 'Polygon', 'coordinates': [[[157.1290...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."


The code below retrieves Sentinel-2 products with cloud cover between 0% and 10%, acquired between June 21, 2021 and September 22, 2021, and located near a specific geographic point:

In [30]:
filters = {
    "cloudCover": "[0,10]",
    "startDate": "2021-06-21T00:00:00Z",
    "completionDate": "2021-09-22T23:59:59Z",
    "lon": 21.01,
    "lat": 52.22,
}

df, resto_url = resto_search("Sentinel2", filters)

print(f"HTTPS Request: {resto_url}")

df

HTTPS Request: https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?cloudCover=%5B0%2C10%5D&startDate=2021-06-21T00%3A00%3A00Z&completionDate=2021-09-22T23%3A59%3A59Z&lon=21.01&lat=52.22


,type,id,geometry,properties
0,Feature,04008903-8704-437b-a8b8-d0942d7fa19c,"{'type': 'Polygon', 'coordinates': [[[22.45053...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
1,Feature,545a3805-d55b-4ce0-b088-3e259072f934,"{'type': 'Polygon', 'coordinates': [[[19.53152...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
2,Feature,f5d8f083-77f3-432a-bd8d-cc816ddd1300,"{'type': 'Polygon', 'coordinates': [[[22.45053...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
3,Feature,8eeb4008-c8e9-4ddc-998c-b22f259d5de6,"{'type': 'Polygon', 'coordinates': [[[19.53152...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
4,Feature,97e26e1a-361d-4e37-b30c-5eba50506187,"{'type': 'Polygon', 'coordinates': [[[21.96038...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
5,Feature,cc1dbd1e-9b1b-4ea0-9b72-08be8bc7ab47,"{'type': 'Polygon', 'coordinates': [[[19.50094...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
6,Feature,ae2a69fd-1ad2-4b20-b9e9-d36b2ec05806,"{'type': 'Polygon', 'coordinates': [[[21.96038...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
7,Feature,83d9e0ae-61f2-4a27-beca-2c428715cf12,"{'type': 'Polygon', 'coordinates': [[[21.96424...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
8,Feature,f09bd83c-7d3a-4823-8e63-b5899fa090b3,"{'type': 'Polygon', 'coordinates': [[[21.96360...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
9,Feature,a3f8f7e9-2276-54e5-b202-dcda07c35839,"{'type': 'Polygon', 'coordinates': [[[20.99970...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."


The code below retrieves up to ten Sentinel-2 products of `S2MSI2A` product type with cloud cover between 0% and 10%, acquired between June 11 and June 22, 2022, and within a geographic bounding box.

In [31]:
filters = {
    "productType": "S2MSI2A",
    "cloudCover": "[0,10]",
    "startDate": "2022-06-11T00:00:00Z",
    "completionDate": "2022-06-22T23:59:59Z",
    "maxRecords": 10,
    "box": "4,51,4.5,52",
}

df, resto_url = resto_search("Sentinel2", filters)

print(f"HTTPS Request: {resto_url}")

df

HTTPS Request: https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?productType=S2MSI2A&cloudCover=%5B0%2C10%5D&startDate=2022-06-11T00%3A00%3A00Z&completionDate=2022-06-22T23%3A59%3A59Z&maxRecords=10&box=4%2C51%2C4.5%2C52


,type,id,geometry,properties
0,Feature,5ecd6532-0200-497d-a7c1-776ff2738d1a,"{'type': 'Polygon', 'coordinates': [[[4.436812...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
1,Feature,5831e264-e17e-4af9-a1f7-d6de71a8a038,"{'type': 'Polygon', 'coordinates': [[[4.436848...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
2,Feature,ef6df284-3d63-4821-8dc2-f86825dfde0f,"{'type': 'Polygon', 'coordinates': [[[4.064087...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
3,Feature,1b32f7ad-ec32-45a1-807b-9adbd510e688,"{'type': 'Polygon', 'coordinates': [[[5.968085...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."
4,Feature,530aeeb3-b889-4596-9c09-3d2e917e9aa4,"{'type': 'Polygon', 'coordinates': [[[4.325619...","{'collection': 'SENTINEL-2', 'status': 'ONLINE..."


The code below retrieves Sentinel-1 products of type `IW_GRDH_1S` with `VV&VH` polarisation:

In [32]:
filters = {"polarisationChannels": "VV&VH", "productType": "IW_GRDH_1S"}

df, resto_url = resto_search("Sentinel1", filters)

print(f"HTTPS Request: {resto_url}")

df

HTTPS Request: https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel1/search.json?polarisationChannels=VV%26VH&productType=IW_GRDH_1S


,type,id,geometry,properties
0,Feature,da70cfa7-8b3b-514a-a807-efd43a2d9652,"{'type': 'Polygon', 'coordinates': [[[17.75591...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
1,Feature,02036627-0bbd-5533-b92f-5229818a9b57,"{'type': 'Polygon', 'coordinates': [[[19.06133...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
2,Feature,9aab3f54-50d9-59f7-8194-e5f0e7539841,"{'type': 'Polygon', 'coordinates': [[[20.65879...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
3,Feature,c7bad712-83bd-5b02-bdfa-be0665ed820a,"{'type': 'Polygon', 'coordinates': [[[2.281085...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
4,Feature,84a3d05a-119e-5d2b-8297-4e23b801361b,"{'type': 'Polygon', 'coordinates': [[[-0.01425...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
5,Feature,bfd85b2c-1d8d-5768-bed5-b5581faed35c,"{'type': 'Polygon', 'coordinates': [[[11.97771...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
6,Feature,69b6f0a6-f5bc-5be0-8899-c0ecab4bccd9,"{'type': 'Polygon', 'coordinates': [[[-9.38965...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
7,Feature,6064f952-7b8f-59c1-919f-18173a6394ad,"{'type': 'Polygon', 'coordinates': [[[84.86653...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
8,Feature,d40fa39b-a849-53e4-99c8-2e1839370628,"{'type': 'Polygon', 'coordinates': [[[46.27384...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."
9,Feature,165d3b85-e65d-5bbe-9cab-f7bad6fee8e7,"{'type': 'Polygon', 'coordinates': [[[42.11571...","{'collection': 'SENTINEL-1', 'status': 'ONLINE..."


### **STAC**

STAC presents a much more standardized metadata model aimed at increasing interoperability and applicability of spatio-temporal datasets. By using a common schema and well-defined object types, STAC makes it easier to discover, understand, and integrate Earth observation data across different providers and missions.

Compared to Resto, STAC offers broader possibilities for data search and filtering. It enables more expressive queries and clearer separation between collections and items, which improves both usability and consistency of the API.

A key concept in STAC is the use of **extensions**, which add additional functionality on top of the core specification. The STAC API specification allows flexible API extensions that enhance core capabilities without changing the base data model.

Currently implemented extensions include:

- `Filter`
- `Query`
- `Fields`
- `Sort`

These extensions significantly extend the search and filtering capabilities of the STAC API while preserving a standardized and interoperable interface.

#### **Filter Extension**

**Filtering** is built around the standardized CQL2 query language, which enables spatial, temporal, and attribute-based comparisons within a single request.

Filtering can be applied to the Item Search endpoint (`/search`) and supports both simple and more complex conditions. Logical operators such as `AND`, `OR`, and `NOT` can be combined with comparison operators (`=`, `<>`, `<`, `<=`, `>`, `>=`) and null checks. These operators can be used with string, numeric, boolean, date, and datetime values. Basic spatial relationships, such as intersection, are also supported.

Queries can be executed using both GET and POST requests. For GET requests, filters are typically expressed using a text-based CQL2 syntax, while POST requests allow filters to be provided in a structured JSON form. This flexibility makes it possible to construct advanced and readable queries while keeping compatibility with standard HTTP workflows.

Detailed and up-to-date information about supported operators, parameters, and query formats is available in the [STAC Catalogue API documentation](https://documentation.dataspace.copernicus.eu/APIs/STAC.html).

In [33]:
stac_url = "https://stac.dataspace.copernicus.eu/v1/search"

In [34]:
params = {
    "filter": {
        "op": "and",
        "args": [
            {"op": "=", "args": [{"property": "collection"}, "sentinel-2-l2a"]},
            {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 10]},
            {
                "op": ">=",
                "args": [
                    {"property": "datetime"},
                    {"timestamp": "2025-04-08T04:39:23Z"},
                ],
            },
            {
                "op": "s_intersects",
                "args": [
                    {"property": "geometry"},
                    {
                        "type": "Polygon",
                        "coordinates": [
                            [
                                [43.5845, -79.5442],
                                [43.6079, -79.4893],
                                [43.5677, -79.4632],
                                [43.6129, -79.3925],
                                [43.6223, -79.3238],
                                [43.6576, -79.3163],
                                [43.7945, -79.1178],
                                [43.8144, -79.1542],
                                [43.8555, -79.1714],
                                [43.7509, -79.6390],
                                [43.5845, -79.5442],
                            ]
                        ],
                    },
                ],
            },
        ],
    }
}

response = requests.post(stac_url, json=params)

json = response.json()

print(
    f"HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&filter=eo:cloud_cover <= 10&datetime >= TIMESTAMP('2025-04-08T04:39:23Z')&S_INTERSECTS(geometry, POLYGON((43.5845 -79.5442, 43.6079 -79.4893, 43.5677 -79.4632, 43.6129 -79.3925, 43.6223 -79.3238, 43.6576 -79.3163, 43.7945 -79.1178, 43.8144 -79.1542, 43.8555 -79.1714, 43.7509 -79.6390, 43.5845 -79.5442)))"
)

df = pd.DataFrame.from_dict(json["features"])
df.head()

HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&filter=eo:cloud_cover <= 10&datetime >= TIMESTAMP('2025-04-08T04:39:23Z')&S_INTERSECTS(geometry, POLYGON((43.5845 -79.5442, 43.6079 -79.4893, 43.5677 -79.4632, 43.6129 -79.3925, 43.6223 -79.3238, 43.6576 -79.3163, 43.7945 -79.1178, 43.8144 -79.1542, 43.8555 -79.1714, 43.7509 -79.6390, 43.5845 -79.5442)))


,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S2B_MSIL2A_20251228T233129_N0511_R101_T38CMS_2...,"[39.72200119694694, -80.25203205661444, 45.516...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[40.19847...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-29T02:20:59.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
1,S2B_MSIL2A_20251228T233129_N0511_R101_T37CEM_2...,"[38.99894187059522, -80.25242038514605, 44.788...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[38.99903...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-29T02:21:40.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
2,S2B_MSIL2A_20251228T000139_N0511_R087_T38CMT_2...,"[40.1593625828737, -79.35014408984915, 44.5605...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[40.29181...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-28T01:34:35.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
3,S2B_MSIL2A_20251228T000139_N0511_R087_T38CMS_2...,"[39.72200119694694, -80.25203205661444, 45.516...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[43.28200...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-28T01:42:28.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
4,S2B_MSIL2A_20251228T000139_N0511_R087_T37CEM_2...,"[38.99894187059522, -80.25242038514605, 44.788...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[43.04873...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-28T01:41:49.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0


#### **Query Extension**

**`Query`** parameter that enables additional filtering based on the properties of Item objects. It allows users to apply simple, property-based conditions when searching for items in the catalog.

The following comparison operators are supported:

- `eq` – equal to  
- `neq` – not equal to  
- `lt` – less than  
- `lte` – less than or equal to  
- `gt` – greater than  
- `gte` – greater than or equal to  

These operators can be used to define constraints on numeric and comparable item properties.

In [35]:
params = {"collections": ["sentinel-2-l2a"], "query": {"eo:cloud_cover": {"lt": 15}}}

response = requests.post(stac_url, json=params)
json = response.json()

print(
    f"HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&eo:cloud_cover%3C15"
)

df = pd.DataFrame.from_dict(json["features"])
df.head()

HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&eo:cloud_cover%3C15


,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S2B_MSIL2A_20251231T051319_N0511_R133_T31CDM_2...,"[-2.277998803053058, -80.22883981478951, 0.184...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-2.18323...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:23:23.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
1,S2B_MSIL2A_20251231T051319_N0511_R133_T30CVS_2...,"[-8.277998803053059, -80.25203205661444, -2.48...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-8.23246...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:25:36.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
2,S2B_MSIL2A_20251231T051319_N0511_R133_T29CNM_2...,"[-9.001058129404774, -80.25242038514605, -3.21...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-9.00104...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:24:16.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
3,S2B_MSIL2A_20251231T051319_N0511_R133_T28CES_2...,"[-13.59074209711865, -80.24048353817358, -9.21...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-13.5907...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:24:35.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
4,S2B_MSIL2A_20251231T050229_N0511_R133_T42FXM_2...,"[70.35704429215036, -48.83293960900481, 71.460...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[70.36265...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:18:51.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0


#### **Fields Extension**

This extension allows users to control which attributes are **included** or **excluded** in search responses. This makes it possible to optimize query performance by **limiting the amount of returned data**, which is especially useful when working with large or complex Item objects.

Through the **`fields`** parameter users can explicitly define which fields should be returned or omitted from the response.

For GET requests, fields can be excluded by prefixing their names with a hyphen. For example, using `-geometry` removes the geometry object from the response. This provides a simple way to reduce response size while keeping only the information relevant to a given workflow.

In [36]:
params = {
    "collections": ["sentinel-2-l2a"],
    "filter": {"op": "lt", "args": [{"property": "eo:cloud_cover"}, 15]},
    "fields": {"exclude": ["geometry"]},
}

response = requests.post(stac_url, json=params)

json = response.json()

print(
    f"HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&filter=eo:cloud_cover%3C15&fields=-geometry"
)

df = pd.DataFrame.from_dict(json["features"])
df.head()

HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&filter=eo:cloud_cover%3C15&fields=-geometry


,id,bbox,type,links,assets,collection,properties,stac_extensions,stac_version
0,S2B_MSIL2A_20251231T051319_N0511_R133_T31CDM_2...,"[-2.277998803053058, -80.22883981478951, 0.184...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:23:23.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
1,S2B_MSIL2A_20251231T051319_N0511_R133_T30CVS_2...,"[-8.277998803053059, -80.25203205661444, -2.48...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:25:36.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
2,S2B_MSIL2A_20251231T051319_N0511_R133_T29CNM_2...,"[-9.001058129404774, -80.25242038514605, -3.21...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:24:16.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
3,S2B_MSIL2A_20251231T051319_N0511_R133_T28CES_2...,"[-13.59074209711865, -80.24048353817358, -9.21...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:24:35.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
4,S2B_MSIL2A_20251231T050229_N0511_R133_T42FXM_2...,"[70.35704429215036, -48.83293960900481, 71.460...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,sentinel-2-l2a,"{'gsd': 10, 'created': '2025-12-31T07:18:51.00...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0


In the following example, the request is further restricted to return only the `assets` of each item.

In [37]:
params = {
    "collections": ["sentinel-2-l2a"],
    "filter": {"op": "lt", "args": [{"property": "eo:cloud_cover"}, 15]},
    "fields": {"include": ["assets"]},
}

response = requests.post(stac_url, json=params)
json = response.json()

print(
    f"HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&filter=eo:cloud_cover%3C15&fields=assets"
)

df = pd.DataFrame.from_dict(json["features"])
df

HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&filter=eo:cloud_cover%3C15&fields=assets


,assets,links
0,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
1,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
2,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
3,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
4,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
5,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
6,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
7,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
8,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."
9,{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"[{'rel': 'collection', 'type': 'application/js..."


#### **Sort Extension**

With this extension users are able to define **sorting** criteria for search results using the **`sortby`** parameter. Sorting can be applied to string, numeric, and datetime fields of an Item or its properties.

Fields can be sorted in either **ascending** or **descending** order. **Multiple fields** may be specified to define a sorting priority.

In GET requests, sorting fields are provided as a comma-separated list. A prefix may be used to indicate the sort order:
- `+` denotes ascending order (default)
- `-` denotes descending order

In POST requests, sorting criteria are expressed as an array, which provides a structured way to define the sorting behavior. Each element of the array specifies a field and the desired order \[`asc` (or `ascending`) and `desc` (or `descending`)].

This extension gives explicit control over result ordering, which is particularly useful when paging through large result sets or comparing items based on specific metadata fields.

In addition to the use of the `sortby` parameter, this query also includes the **`limit`** parameter, which defines the maximum number of Items to be returned in the response. The default value is set to 12.

In [38]:
params = {
    "collections": ["sentinel-2-l2a"],
    "query": {"eo:cloud_cover": {"gt": 10}},
    "datetime": "2025-01-01T00:00:00Z/2025-01-15T23:59:59Z",
    "sortby": [{"field": "eo:cloud_cover", "direction": "asc"}],
    "limit": 10,
}

response = requests.post(stac_url, json=params)

json = response.json()

print(
    f"HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items?filter=eo:cloud_cover%3C15&datetime=2025-01-01T00:00:00Z/2025-01-15T23:59:59Z&sortby=+properties.eo:cloud_cover&limit=10"
)

df = pd.DataFrame.from_dict(json["features"])
df

HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items?filter=eo:cloud_cover%3C15&datetime=2025-01-01T00:00:00Z/2025-01-15T23:59:59Z&sortby=+properties.eo:cloud_cover&limit=10


,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S2B_MSIL2A_20250115T043029_N0511_R133_T45QXD_2...,"[88.664444, 20.851601, 89.027861, 21.692646]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[89.01644...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-15T07:22:11.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
1,S2B_MSIL2A_20250114T190459_N0511_R127_T05CMS_2...,"[-156.43536, -74.863265, -156.220227, -74.762472]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-156.220...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-14T23:33:48.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
2,S2B_MSIL2A_20250112T160529_N0511_R097_T17QLB_2...,"[-82.910307, 18.894431, -82.404586, 19.890114]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-82.4045...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-12T21:56:54.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
3,S2B_MSIL2A_20250111T215909_N0511_R086_T02LKJ_2...,"[-173.79563, -15.450678, -173.254328, -14.455833]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-173.254...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-11T23:29:04.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
4,S2B_MSIL2A_20250108T194719_N0511_R042_T10VEN_2...,"[-123.000374, 60.33627, -120.949436, 61.334442]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[-120.983...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-08T23:35:08.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
5,S2B_MSIL2A_20250101T204019_N0511_R085_T45CWK_2...,"[86.99875, -82.012669, 94.069006, -81.007392]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[90.74373...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T22:39:28.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
6,S2A_MSIL2A_20250106T000731_N0511_R073_T57MTR_2...,"[156.297745, -4.122706, 156.409008, -3.61448]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[156.4090...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-06T02:48:01.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
7,S2A_MSIL2A_20250104T055231_N0511_R048_T43SBT_2...,"[71.754262, 33.309546, 72.956153, 34.324206]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[71.75426...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-04T09:50:28.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
8,S2B_MSIL2A_20250114T100259_N0511_R122_T32RQV_2...,"[11.086289, 30.62773, 11.392596, 31.618143]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[11.39259...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-14T13:23:07.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
9,S2B_MSIL2A_20250113T085229_N0511_R107_T34LBL_2...,"[18.227016, -13.641674, 18.47779, -12.649679]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[18.47779...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-13T12:09:51.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0


To verify that the query sorts the results correctly, we can inspect the cloud cover values of the returned products.

In [39]:
df_cc = pd.json_normalize(json["features"], sep=".")
df_cc[["id", "properties.eo:cloud_cover"]]

,id,properties.eo:cloud_cover
0,S2B_MSIL2A_20250115T043029_N0511_R133_T45QXD_2...,10.01
1,S2B_MSIL2A_20250114T190459_N0511_R127_T05CMS_2...,10.01
2,S2B_MSIL2A_20250112T160529_N0511_R097_T17QLB_2...,10.01
3,S2B_MSIL2A_20250111T215909_N0511_R086_T02LKJ_2...,10.01
4,S2B_MSIL2A_20250108T194719_N0511_R042_T10VEN_2...,10.01
5,S2B_MSIL2A_20250101T204019_N0511_R085_T45CWK_2...,10.01
6,S2A_MSIL2A_20250106T000731_N0511_R073_T57MTR_2...,10.01
7,S2A_MSIL2A_20250104T055231_N0511_R048_T43SBT_2...,10.01
8,S2B_MSIL2A_20250114T100259_N0511_R122_T32RQV_2...,10.02
9,S2B_MSIL2A_20250113T085229_N0511_R107_T34LBL_2...,10.02


#### **Resto query equivalents in STAC**

With the concepts introduced above, we can now attempt to translate selected Resto queries into their STAC equivalents. This section demonstrates how typical filtering patterns used in the Resto can be remapped to STAC search requests, highlighting the differences in syntax while preserving the original query intent.

In [40]:
stac_url = "https://stac.dataspace.copernicus.eu/v1/search"

This code queries the Sentinel-2 L2A collection for items from January 1 to January 31, 2025, using the collection name, datetime range, a limit of 20 results, and sorting by datetime in ascending order.

In [41]:
params = {
    "collections": ["sentinel-2-l2a"],
    "datetime": "2025-01-01T00:00:00Z/2025-01-31T23:59:59Z",
    "limit": 20,
    "sortby": [{"field": "datetime", "direction": "asc"}],
}

response = requests.post(stac_url, json=params)

json = response.json()

print(
    f"HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&datetime=2025-01-01T00:00:00Z/2025-01-31T23:59:59Z&limit=20&sortby=+datetime"
)

df_features = pd.DataFrame(json["features"])
df_features

HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&datetime=2025-01-01T00:00:00Z/2025-01-31T23:59:59Z&limit=20&sortby=+datetime


,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S2B_MSIL2A_20250101T000439_N0511_R073_T57NVH_2...,"[158.093982, 6.243793, 158.745574, 7.155754]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[158.0939...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:44:17.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
1,S2B_MSIL2A_20250101T000439_N0511_R073_T57NVG_2...,"[158.095521, 5.339213, 158.599385, 6.33265]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[158.2868...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:30:22.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
2,S2B_MSIL2A_20250101T000439_N0511_R073_T57NVF_2...,"[158.096981, 5.31548, 158.396634, 5.427753]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[158.3183...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:23:15.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
3,S2B_MSIL2A_20250101T000439_N0511_R073_T57NUJ_2...,"[157.188064, 7.145259, 158.12463, 7.360241]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[157.1880...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:27:52.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
4,S2B_MSIL2A_20250101T000439_N0511_R073_T57NUH_2...,"[157.188619, 6.241466, 158.184617, 7.235884]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[157.7636...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:47:52.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
5,S2B_MSIL2A_20250101T000439_N0511_R073_T57NUG_2...,"[157.191994, 5.367592, 158.185873, 6.332402]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[158.1858...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:33:24.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
6,S2B_MSIL2A_20250101T000439_N0511_R073_T57NUF_2...,"[157.949753, 5.367593, 158.185869, 5.427677]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[158.1858...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:24:35.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
7,S2B_MSIL2A_20250101T000439_N0511_R073_T57NTJ_2...,"[156.281374, 7.140815, 157.277502, 7.532774]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[156.2813...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:28:04.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
8,S2B_MSIL2A_20250101T000439_N0511_R073_T57NTH_2...,"[156.283325, 6.237589, 157.280668, 7.234569]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[156.2833...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:36:42.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
9,S2B_MSIL2A_20250101T000439_N0511_R073_T57NTG_2...,"[156.288384, 5.591788, 157.282626, 6.330194]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[157.2826...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-01-01T01:44:40.00...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0


This code queries the Sentinel-2 L2A collection for items between June 21 and September 22, 2021, that intersect the point at coordinates [21.01, 52.22] and have a cloud cover between 0% and 10%.

In [42]:
params = {
    "collections": ["sentinel-2-l2a"],
    "datetime": "2021-06-21T00:00:00Z/2021-09-22T23:59:59Z",
    "intersects": {"type": "Point", "coordinates": [21.01, 52.22]},
    "query": {"eo:cloud_cover": {"gte": 0, "lte": 10}},
}

response = requests.post(stac_url, json=params)

json = response.json()

print(
    f"HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&datetime=2021-06-21T00%3A00%3A00Z%2F2021-09-22T23%3A59%3A59Z&intersects=%7B%22type%22%3A%22Point%22%2C%22coordinates%22%3A%5B21.01%2C52.22%5D%7D&filter=eo%3Acloud_cover%20%3C%3D%2010"
)

df_features = pd.DataFrame(json["features"])
df_features

HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&datetime=2021-06-21T00%3A00%3A00Z%2F2021-09-22T23%3A59%3A59Z&intersects=%7B%22type%22%3A%22Point%22%2C%22coordinates%22%3A%5B21.01%2C52.22%5D%7D&filter=eo%3Acloud_cover%20%3C%3D%2010


,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S2B_MSIL2A_20210909T095029_N0500_R079_T34UEC_2...,"[20.999706, 51.360232, 21.963602, 52.350473]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[21.96360...",sentinel-2-l2a,"{'gsd': 10, 'created': '2023-03-13T01:48:59.09...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
1,S2B_MSIL2A_20210909T095029_N0500_R079_T34UDC_2...,"[19.531529, 51.354432, 21.143291, 52.350386]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[19.53152...",sentinel-2-l2a,"{'gsd': 10, 'created': '2023-03-13T01:53:31.43...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
2,S2A_MSIL2A_20210904T095031_N0500_R079_T34UEC_2...,"[20.999706, 51.360262, 21.960142, 52.350473]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[21.96014...",sentinel-2-l2a,"{'gsd': 10, 'created': '2023-03-12T16:22:37.90...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
3,S2B_MSIL2A_20210807T094029_N0500_R036_T34UEC_2...,"[20.999706, 51.352634, 22.611384, 52.350473]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[20.99970...",sentinel-2-l2a,"{'gsd': 10, 'created': '2023-02-14T13:47:12.87...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
4,S2B_MSIL2A_20210711T095029_N0500_R079_T34UEC_2...,"[20.999706, 51.360286, 21.955875, 52.350473]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[21.95587...",sentinel-2-l2a,"{'gsd': 10, 'created': '2023-03-15T08:23:16.75...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
5,S2B_MSIL2A_20210708T094029_N0500_R036_T34UEC_2...,"[20.999706, 51.352634, 22.611384, 52.350473]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[20.99970...",sentinel-2-l2a,"{'gsd': 10, 'created': '2023-03-16T17:59:14.58...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
6,S2B_MSIL2A_20210621T095029_N0500_R079_T34UEC_2...,"[20.999706, 51.360262, 21.960388, 52.350473]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[21.96038...",sentinel-2-l2a,"{'gsd': 10, 'created': '2023-06-10T22:54:38.64...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0
7,S2B_MSIL2A_20210621T095029_N0500_R079_T34UDC_2...,"[19.531529, 51.354432, 21.143291, 52.350386]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[19.53152...",sentinel-2-l2a,"{'gsd': 10, 'created': '2023-06-10T22:58:56.96...",[https://stac-extensions.github.io/eo/v2.0.0/s...,1.1.0


This code queries the Sentinel-2 L2A collection for items between June 11 and June 22, 2022, within the bounding box [4, 51, 4.5, 52], with cloud cover between 0% and 10%, limiting the results to 10 items.

In [43]:
params = {
    "collections": ["sentinel-2-l2a"],
    "datetime": "2022-06-11T00:00:00Z/2022-06-22T23:59:59Z",
    "bbox": [4, 51, 4.5, 52],
    "limit": 10,
    "query": {"eo:cloud_cover": {"gte": 0, "lte": 10}},
}

response = requests.post(stac_url, json=params)

json = response.json()

print(
    f"HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&datetime=2022-06-11T00%3A00%3A00Z%2F2022-06-22T23%3A59%3A59Z&bbox=4,51,4.5,52&filter=eo%3Acloud_cover%20%3E%3D%200%20AND%20eo%3Acloud_cover%20%3C%3D%2010&limit=10"
)

df_features = pd.DataFrame(json["features"])
df_features

HTTPS GET Request: https://stac.dataspace.copernicus.eu/v1/search?collections=sentinel-2-l2a&datetime=2022-06-11T00%3A00%3A00Z%2F2022-06-22T23%3A59%3A59Z&bbox=4,51,4.5,52&filter=eo%3Acloud_cover%20%3E%3D%200%20AND%20eo%3Acloud_cover%20%3C%3D%2010&limit=10


,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S2A_MSIL2A_20220622T105631_N0510_R094_T31UET_2...,"[2.999706370881351, 51.357799703919795, 4.3256...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[4.325619...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-02-15T08:31:15.80...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
1,S2A_MSIL2A_20220616T103631_N0510_R008_T31UFT_2...,"[4.436120625123461, 51.32452335478603, 6.07775...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[4.436848...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-02-14T19:06:43.50...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
2,S2B_MSIL2A_20220614T104629_N0510_R051_T31UFS_2...,"[4.408715109637645, 50.42809119252208, 6.01702...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[5.968085...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-02-14T14:34:10.93...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
3,S2B_MSIL2A_20220611T103629_N0510_R008_T31UFS_2...,"[4.408715109637645, 50.42629366061962, 6.01702...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[4.436812...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-02-14T08:11:38.91...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
4,S2B_MSIL2A_20220611T103629_N0510_R008_T31UES_2...,"[4.064087128317525, 50.453523278919015, 4.5795...",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'AOT_10m': {'href': 's3://eodata/Sentinel-2/M...,"{'type': 'Polygon', 'coordinates': [[[4.064087...",sentinel-2-l2a,"{'gsd': 10, 'created': '2025-02-14T08:11:38.75...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0


This code uses the query parameter to search the Sentinel-1 GRD collection for items where the product type is `IW_GRDH_1S` and the polarization is `VV&VH`, returning up to 10 results.

In [44]:
params = {
    "collections": ["sentinel-1-grd"],
    "query": {
        "product:type": {"eq": "IW_GRDH_1S"},
        "sar:polarizations": {"eq": ["VV", "VH"]},
    },
    "limit": 10,
}

response = requests.post(stac_url, json=params)

json = response.json()


df_features = pd.DataFrame(json["features"])
df_features

,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S1A_IW_GRDH_1SDV_20251231T061257_20251231T0613...,"[-4.961516, 29.132935, -2.043568, 30.6371]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-2.28483...",sentinel-1-grd,"{'created': '2025-12-31T06:01:17.880940Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
1,S1A_IW_GRDH_1SDV_20251231T061232_20251231T0612...,"[-4.750492, 30.215359, -1.708874, 32.141827]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-2.04354...",sentinel-1-grd,"{'created': '2025-12-31T06:01:18.350244Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
2,S1A_IW_GRDH_1SDV_20251231T061207_20251231T0612...,"[-4.460252, 31.722683, -1.369323, 33.646049]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-1.70885...",sentinel-1-grd,"{'created': '2025-12-31T06:12:00.285895Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
3,S1A_IW_GRDH_1SDV_20251231T061142_20251231T0612...,"[-4.168624, 33.229343, -1.009401, 35.147907]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-1.3693,...",sentinel-1-grd,"{'created': '2025-12-31T06:12:01.441023Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
4,S1A_IW_GRDH_1SDV_20251231T061117_20251231T0611...,"[-3.860411, 34.733044, -0.639349, 36.648712]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-1.00938...",sentinel-1-grd,"{'created': '2025-12-31T06:11:58.380623Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
5,S1A_IW_GRDH_1SDV_20251231T061052_20251231T0611...,"[-3.545959, 36.235367, -0.277303, 38.150902]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-0.63932...",sentinel-1-grd,"{'created': '2025-12-31T06:11:58.937265Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
6,S1A_IW_GRDH_1SDV_20251231T061027_20251231T0610...,"[-3.243545, 37.739258, 0.093031, 39.652462]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-0.27728...",sentinel-1-grd,"{'created': '2025-12-31T06:11:47.595395Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
7,S1A_IW_GRDH_1SDV_20251231T061002_20251231T0610...,"[-2.937376, 39.242229, 0.463011, 41.154549]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[0.093054...",sentinel-1-grd,"{'created': '2025-12-31T06:11:48.024200Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
8,S1A_IW_GRDH_1SDV_20251231T060937_20251231T0610...,"[-2.63635, 40.745724, 0.852503, 42.654728]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[0.463035...",sentinel-1-grd,"{'created': '2025-12-31T06:12:00.578034Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
9,S1A_IW_GRDH_1SDV_20251231T060912_20251231T0609...,"[-2.321066, 42.246735, 1.270123, 44.15239]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[0.852526...",sentinel-1-grd,"{'created': '2025-12-31T06:11:59.740213Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0


This code uses the filter parameter to search the Sentinel-1 GRD collection for items where the product type is `IW_GRDH_1S` and the polarization is `VV&VH`, returning up to 10 results.

In [45]:
params = {
    "filter-lang": "cql2-json",
    "filter": {
        "op": "and",
        "args": [
            {"op": "=", "args": [{"property": "product:type"}, "IW_GRDH_1S"]},
            {"op": "a_contains", "args": [{"property": "sar:polarizations"}, "VV"]},
            {"op": "a_contains", "args": [{"property": "sar:polarizations"}, "VH"]},
        ],
    },
    "collections": ["sentinel-1-grd"],
    "limit": 10,
}

response = requests.post(stac_url, json=params)

json = response.json()

df_features = pd.DataFrame(json["features"])
df_features

,id,bbox,type,links,assets,geometry,collection,properties,stac_extensions,stac_version
0,S1A_IW_GRDH_1SDV_20251231T061257_20251231T0613...,"[-4.961516, 29.132935, -2.043568, 30.6371]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-2.28483...",sentinel-1-grd,"{'created': '2025-12-31T06:01:17.880940Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
1,S1A_IW_GRDH_1SDV_20251231T061232_20251231T0612...,"[-4.750492, 30.215359, -1.708874, 32.141827]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-2.04354...",sentinel-1-grd,"{'created': '2025-12-31T06:01:18.350244Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
2,S1A_IW_GRDH_1SDV_20251231T061207_20251231T0612...,"[-4.460252, 31.722683, -1.369323, 33.646049]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-1.70885...",sentinel-1-grd,"{'created': '2025-12-31T06:12:00.285895Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
3,S1A_IW_GRDH_1SDV_20251231T061142_20251231T0612...,"[-4.168624, 33.229343, -1.009401, 35.147907]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-1.3693,...",sentinel-1-grd,"{'created': '2025-12-31T06:12:01.441023Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
4,S1A_IW_GRDH_1SDV_20251231T061117_20251231T0611...,"[-3.860411, 34.733044, -0.639349, 36.648712]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-1.00938...",sentinel-1-grd,"{'created': '2025-12-31T06:11:58.380623Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
5,S1A_IW_GRDH_1SDV_20251231T061052_20251231T0611...,"[-3.545959, 36.235367, -0.277303, 38.150902]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-0.63932...",sentinel-1-grd,"{'created': '2025-12-31T06:11:58.937265Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
6,S1A_IW_GRDH_1SDV_20251231T061027_20251231T0610...,"[-3.243545, 37.739258, 0.093031, 39.652462]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[-0.27728...",sentinel-1-grd,"{'created': '2025-12-31T06:11:47.595395Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
7,S1A_IW_GRDH_1SDV_20251231T061002_20251231T0610...,"[-2.937376, 39.242229, 0.463011, 41.154549]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[0.093054...",sentinel-1-grd,"{'created': '2025-12-31T06:11:48.024200Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
8,S1A_IW_GRDH_1SDV_20251231T060937_20251231T0610...,"[-2.63635, 40.745724, 0.852503, 42.654728]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[0.463035...",sentinel-1-grd,"{'created': '2025-12-31T06:12:00.578034Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0
9,S1A_IW_GRDH_1SDV_20251231T060912_20251231T0609...,"[-2.321066, 42.246735, 1.270123, 44.15239]",Feature,"[{'rel': 'collection', 'type': 'application/js...",{'vh': {'href': 's3://eodata/Sentinel-1/SAR/IW...,"{'type': 'Polygon', 'coordinates': [[[0.852526...",sentinel-1-grd,"{'created': '2025-12-31T06:11:59.740213Z', 'ex...",[https://cs-si.github.io/eopf-stac-extension/v...,1.1.0


# **Conclusion**

This notebook highlights the fundamental differences between the OpenSearch (Resto) Catalogue API and the STAC Catalogue API, with a strong focus on discovery capabilities and query expressiveness. Resto provides a relatively flat and limited search interface, primarily centered around collection-level queries with a fixed set of parameters and constrained filtering logic. While sufficient for basic discovery use cases, this model restricts fine-grained access to metadata and limits the ability to precisely target individual products or their components.

In contrast, STAC introduces a hierarchical and standardized data model that enables discovery at multiple levels, ranging from collections, through individual items, down to specific assets within an item. This multi-level access is combined with a significantly richer querying framework. Through extensions such as Query, Filter, Fields, and Sort, STAC allows users to construct complex spatial, temporal, and attribute-based queries, apply precise sorting rules, limit result sets, and control the structure and size of responses. These capabilities make it possible to efficiently narrow searches to highly specific subsets of data, optimize performance, and integrate discovery workflows more tightly with downstream processing.

Thanks to this notebook and its step-by-step examples, understanding the STAC data model and transitioning existing workflows from OpenSearch to STAC should be straightforward and significantly easier to implement in practice.

Overall, migrating from Resto to STAC is not merely a change of endpoint but a shift to a more powerful, extensible, and interoperable discovery paradigm. STAC enables more precise data selection, better scalability, and improved integration with modern geospatial and data processing ecosystems, making it a robust foundation for future Earth Observation data access and exploitation.